In [ ]:
from keras.utils import to_categorical
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Dense, Input, Embedding, MaxPooling1D, Flatten
from keras.callbacks import ModelCheckpoint
import numpy as np
import pickle
from matplotlib import image, pyplot
from skimage.transform import resize
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
from keras import callbacks
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import os
import keras
import tensorflow_addons as tfa
from xgboost import XGBClassifier

DATA_PATH = 'image/'
spamData=[]
label=[]
hashList=[]
count=-1
for folder in os.listdir(DATA_PATH):
    if folder=="New_Spam":
        continue;
    print(">>>Reading ",folder)
    count+=1
    
    for file in os.listdir(DATA_PATH+folder):
        if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in hashList):
                spamData.append(resize(img, (32, 32, 3)))
                hashList.append(hsh)
                label.append(count)
spamData=np.array(spamData)
label=np.array(label)

resultPath = 'result/'
train_folder = os.listdir(DATA_PATH)

In [ ]:
print("Number of SPAM",len(label[label==0]))
print("Number of HAM",len(label[label==1]))

print("Spam data shape : ",spamData.shape," Label shape : ",label.shape)

In [ ]:
test_valid_Percentage = 0.3
x_train,x_test,y_train,y_test = train_test_split(spamData,label,test_size = test_valid_Percentage,random_state=42, stratify=label,shuffle=True)

print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)

In [ ]:
print("Number of train SPAM",len(y_train[y_train==0]))
print("Number of train HAM",len(y_train[y_train==1]))

print("Number of test SPAM",len(y_test[y_test==0]))
print("Number of test HAM",len(y_test[y_test==1]))

In [ ]:
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

print("y_train_oh shape : ",y_train_oh.shape," y_test_oh shape : ",y_test_oh.shape)

In [ ]:
model=Sequential()
model.add(Conv2D(32,kernel_size=3, padding='same',activation='leaky_relu',input_shape=(32, 32,3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64,kernel_size=3, padding='same',activation='leaky_relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128,kernel_size=3, padding='same',activation='leaky_relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128,activation='leaky_relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
xgbmodel = XGBClassifier(objective='multi:softprob', num_class= 10)
xgbmodel.fit(x_train, y_train)
xgbmodel.score(val_data, val_label1)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), tfa.metrics.F1Score(num_classes=1)])

In [ ]:
history = model.fit(x_train, y_train, epochs=10, verbose=1, batch_size=16, validation_data=(x_test,y_test))

In [ ]:
results = ['name', 'Accuracy', 'Precision', 'Recall', 'F1-Score']

results += ['DeepCapture', history.history['val_accuracy'][5], history.history['val_precision'][5], history.history['val_recall'][5], history.history['val_f1_score'][5][0]]


print('________________________________________________________________________________________________________________')
print("{0}\t\t{1}s\t\t{2}\t\t{3}\t\t\t{4}s\
      \n\n{5}\t{6}\t{7}\t{8}\t{9}s".format(*results))
print('________________________________________________________________________________________________________________')